## Implementing Reflexion Agent

### Actor Agent

#### Schema

In [93]:
# Imports
from pydantic import BaseModel, Field
from typing import List

In [94]:
class Reflection(BaseModel):
    missing: str = Field(description="Critique of what is missing.")
    superfluous: str = Field(description="Critique of what is superfluous.")

In [95]:
class AnswerQuestion(BaseModel):
    """Answer the questions"""
    answer: str = Field(description="250 word detailed answer to the question.")
    reflection: Reflection = Field(description="Your reflection on the initial answer.")
    queries: List[str] = Field(description="1-3 search queries for researching improvements to address the critique of your current answer.")

#### Chains

In [96]:
#imports
from datetime import datetime
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import JsonOutputToolsParser, PydanticToolsParser
from langchain_core.messages import HumanMessage


In [127]:
load_dotenv()
llm = ChatGroq(model="llama-3.3-70b-versatile")
llm=llm.bind_tools(tools=[AnswerQuestion],tool_choice="AnswerQuestion")

In [128]:
json_parser = JsonOutputToolsParser(return_id=True)
pydantic_parser = PydanticToolsParser(tools=[AnswerQuestion])

In [129]:
actor_prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system",""" 
                    You are expert researcher.
                    Current time: {time}

                    1. {first_instruction}
                    2. Reflect and critique your answer. Be severe to maximize improvement.
                    3. Recommend search queries to research information and improve your answer.
                 """),
        MessagesPlaceholder(variable_name="messages"),
        ("system","Answer the user's question above using the required format.")
    ]
).partial(time=lambda: datetime.now().isoformat())

In [130]:
first_responder_prompt_template = actor_prompt_template.partial(first_instruction="Provide a detailed 200 word answer.")

In [131]:
human_message = HumanMessage(
    content="Write about AI-Powered SOC / autonomous soc  problem domain, " \
    "list startups that do that and raised capital.")

In [132]:
chain = (first_responder_prompt_template|llm|pydantic_parser)

In [133]:
response = chain.invoke(input={"messages":[human_message]})

In [134]:
from pprint import pprint
pprint(response[0].answer)
pprint(response[0].reflection)
pprint(response[0].queries)

('The AI-Powered Security Operations Center (SOC) or autonomous SOC is a '
 'problem domain that involves the use of artificial intelligence and machine '
 'learning to automate and enhance the capabilities of traditional security '
 'operations centers. The goal of an AI-Powered SOC is to improve the '
 'efficiency and effectiveness of security operations by leveraging AI and ML '
 'to detect, respond, and prevent cyber threats in real-time. This is achieved '
 'through the use of advanced analytics, automation, and orchestration of '
 'security incident response processes. Startups such as Deep Instinct, '
 'Cybereason, and SentinelOne are working in this space and have raised '
 'significant capital to develop and deploy their solutions. For example, Deep '
 'Instinct has raised over $100 million in funding to develop its AI-powered '
 'cybersecurity platform, while Cybereason has raised over $300 million to '
 'develop its AI-powered SOC platform. Other startups such as Automate '
